<a href="https://colab.research.google.com/github/wjdqlsdlsp/Quant_Project-Solo--/blob/main/Kospi_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install finance-datareader

In [ ]:
pip install plotly==4.14.3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import FinanceDataReader as fdr

In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

In [ ]:
# Data loading
df = pd.read_csv("/content/data_5519_20210319.csv",encoding = 'cp949')
stocks = fdr.StockListing('KRX')

In [ ]:
all = df[(df['시장구분'] =='KOSPI') | (df['시장구분']=='KOSDAQ')]
kospi = df[(df['시장구분'] =='KOSPI')]
not_use_columns = ["소속부","종가","대비","시가","고가","저가","거래량","거래대금","상장주식수"]
kospi = kospi.drop(not_use_columns, axis=1)
sector = stocks[["Symbol","Sector"]]
sector = sector.to_numpy()
def make_sector(code):
  for i in range(len(sector)):
    if code == sector[i][0]:
      return sector[i][1]
  return 'No'

kospi["섹터"] = kospi["종목코드"].map(make_sector)
kospi = kospi.fillna('우선주')

In [ ]:
fig = px.treemap(kospi, path=['시장구분','섹터','종목명'], values='시가총액',
                  color='등락률', hover_data=['등락률','종목코드'],
                 range_color=(-0.3,0.3),color_continuous_scale='balance')
fig.update_traces(textposition='middle center',text = kospi['등락률'])
fig.update_traces(textfont_color='white',textfont_size=25)
fig.update_layout(height=1000)
fig.update_layout(coloraxis_showscale=False)
fig.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

usdkrw = fdr.DataReader('USD/KRW', '2020-01-01')
index = usdkrw.index
column = usdkrw.columns
scaler = MinMaxScaler()
usdkrw = pd.DataFrame(scaler.fit_transform(usdkrw),index=index, columns=column)

kospi_index = fdr.DataReader('KS11', '2020-01-01')
index = kospi_index.index
column = kospi_index.columns
scaler = MinMaxScaler()
kospi_index = pd.DataFrame(scaler.fit_transform(kospi_index),index=index, columns=column)

UStnx = fdr.DataReader('US10YT=X','2020-01-01')
index = UStnx.index
column = UStnx.columns
scaler = MinMaxScaler()
UStnx = pd.DataFrame(scaler.fit_transform(UStnx),index=index, columns=column)

highyield = pd.read_csv("/content/BAMLH0A0HYM2EY.csv")
highyield = highyield.set_index('DATE')
for i in highyield.index:
  if highyield.loc[i,'BAMLH0A0HYM2EY'] =='.':
    highyield.drop(i,inplace=True)

pd.to_numeric(highyield['BAMLH0A0HYM2EY'])

index = highyield.index
column = highyield.columns
scaler = MinMaxScaler()
highyield = pd.DataFrame(scaler.fit_transform(highyield),index=index, columns=column)

wti = pd.read_csv("/content/CL=F.csv")
wti.set_index("Date",inplace=True)
index = wti.index
column = wti.columns
scaler = MinMaxScaler()
wti = pd.DataFrame(scaler.fit_transform(wti),index=index, columns=column)

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=kospi_index.index,
                open=kospi_index['Open'],
                high=kospi_index['High'],
                low=kospi_index['Low'],
                close=kospi_index['Close'],
                increasing_line_color= 'red',
                decreasing_line_color= 'blue',
                name='Kospi')])
fig.add_trace(go.Scatter(x=usdkrw.index, y=usdkrw['Close'],name='USDKRW'))
fig.add_trace(go.Scatter(x=UStnx.index, y=UStnx['Close'],name='UStnx'))
fig.add_trace(go.Scatter(x=highyield.index, y=highyield['BAMLH0A0HYM2EY'],name='highyield'))
fig.add_trace(go.Scatter(x=wti.index, y=wti['Close'],name='WTI'))

fig.show()